First, import needed modules

In [ ]:
import numpy as np
import pandas as pd
import os

Initalize variables

In [ ]:
path = os.getcwd()
print(path)

Import datasets

In [ ]:
btw02_es = pd.read_csv(f'{path}/data/elections/btw/es/BTW02_Erststimmen_Wahlbezirke.csv', encoding = 'UTF-8-SIG', sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Verbandsgemeinde': str, 'Gemeinde': str})
btw05_es = pd.read_csv(f'{path}/data/elections/btw/es/BTW05_Erststimmen_Wahlbezirke.csv', encoding = 'UTF-8-SIG',sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str,  'Verbandsgemeinde': str,'Gemeinde': str})
btw09_es = pd.read_csv(f'{path}/data/elections/btw/es/BTW09_Erststimmen_Wbz.csv', sep=';', encoding = 'UTF-8-SIG',skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Verbandsgemeinde': str,'Gemeinde': str})
btw13_es = pd.read_csv(f'{path}/data/elections/btw/es/BTW13_Erststimmen_Wahlbezirke.csv', encoding = 'UTF-8-SIG',sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Verbandsgemeinde': str,'Gemeinde': str})
btw17_es = pd.read_csv(f'{path}/data/elections/btw/es/btw17_wbz_erststimmen.csv', sep=';', encoding = 'Windows-1252', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Verbandsgemeinde': str,'Gemeinde': str})

btw02_zs = pd.read_csv(f'{path}/data/elections/btw/zs/BTW02_Zweitstimmen_Wahlbezirke.csv', encoding = 'UTF-8-SIG', sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Verbandsgemeinde': str,'Gemeinde': str})
btw05_zs = pd.read_csv(f'{path}/data/elections/btw/zs/BTW05_Zweitstimmen_Wahlbezirke.csv', encoding = 'UTF-8-SIG',sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Verbandsgemeinde': str,'Gemeinde': str})
btw09_zs = pd.read_csv(f'{path}/data/elections/btw/zs/BTW09_Zweitstimmen_Wbz.csv', sep=';', encoding = 'UTF-8-SIG',skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Verbandsgemeinde': str,'Gemeinde': str})
btw13_zs = pd.read_csv(f'{path}/data/elections/btw/zs/BTW13_Zweitstimmen_Wahlbezirke.csv', encoding = 'UTF-8-SIG',sep=';', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Verbandsgemeinde': str,'Gemeinde': str})
btw17_zs = pd.read_csv(f'{path}/data/elections/btw/zs/btw17_wbz_zweitstimmen.csv', sep=';', encoding = 'Windows-1252', skiprows=4, converters = {'Land': str, 'Regierungsbezirk': str, 'Kreis': str, 'Verbandsgemeinde': str,'Gemeinde': str})

View the data

In [ ]:
btw09_zs.head()

Add year column to dataframes

In [ ]:
dfs = [btw02_es, btw05_es, btw09_es, btw13_es, btw17_es, btw02_zs, btw05_zs, btw09_zs, btw13_zs, btw17_zs]
years = [2002, 2005, 2009, 2013, 2017, 2002, 2005, 2009, 2013, 2017]
for df, year in zip(dfs, years):
    df.insert(loc=len(df.columns), column='year', value=year)

In [ ]:
btw17_es.head()

Append dataframes

In [ ]:
dfs = [btw02_es, btw05_es, btw09_es, btw13_es, btw17_es]
btw_es = pd.concat(dfs, sort=False)
dfs = [btw02_zs, btw05_zs, btw09_zs, btw13_zs, btw17_zs]
btw_zs = pd.concat(dfs, sort=False)
btw_zs.head()

Create the AGS, it follows the scheme ll r kk ggg (Land, Regierunsbezirk, Kreis, Gemeinde) -> combine these columns

In [ ]:
btw_es['RS'] = btw_es['Land'] + btw_es['Regierungsbezirk'] + btw_es['Kreis'] + btw_es['Verbandsgemeinde'] + btw_es['Gemeinde']
btw_zs['RS'] = btw_zs['Land'] + btw_zs['Regierungsbezirk'] + btw_zs['Kreis'] + btw_es['Verbandsgemeinde'] + btw_zs['Gemeinde']
btw_zs.head()

Clean dataset

In [ ]:
# collapse data along AGS column, sums votes over all constituencies in a municipality
btw_es = btw_es.groupby(['RS', 'year','Land'], as_index=False).sum()
btw_zs = btw_zs.groupby(['RS', 'year','Land'], as_index=False).sum()
btw_zs.head()

Add indicator for Erst- und Zweitstimme

In [ ]:
dfs = [btw_es, btw_zs]
eszs = ['Erststimme', 'Zweitstimme']
for df, stimme in zip(dfs, eszs):
    df.insert(loc=len(df.columns), column=stimme, value=1)

In [ ]:
btw_zs.head()

ES: Sum votes for other, Union and Die Linke parties

In [ ]:
#list(btw_es.columns)

In [ ]:
andere = ['AfD', 
 'REP',
 'GRAUE',
 'TIERSCHUTZ',
 'NPD',
 'ÖDP',
 'PBC',
 'FRAUEN',
 'BP',
 'FAMILIE',
 'CM',
 'BÜSO',
 'HP',
 'VIOLETTEN',
 'AUFBRUCH',
 'ZENTRUM',
 'KPD',
 'PRG',
 'SCHILL',
 'DEUTSCHLAND',
 'DKP',
 'DSU',
 'FP D',
 'ÜBRIGE',
 'Die Linke',
 'Offensive D',
 'Die Tierschutzpartei',
 'DIE FRAUEN',
 'BüSo',
 'Deutschland',
 'AGFG',
 'APPD',
 'MLPD',
 'DIE PARTEI',
 'HUMANWIRTSCHAFTSPARTEI',
 'STATT Partei',
 'UNABHÄNGIGE',
 'Übrige',
 'DIE LINKE',
 'Volksabstimmung',
 'ADM',
 'DIE VIOLETTEN',
 'ödp',
 'PIRATEN',
 'RRP',
 'Freie Union',
 'Tierschutzpartei',
 'Bündnis 21/RRP',
 'RENTNER',
 'BIG',
 'pro Deutschland',
 'FREIE WÄHLER',
 'PARTEI DER VERNUNFT',
 'Die PARTEI',
 'Bergpartei',
 'BGD',
 'NEIN',
 'WGr/EB',
 'PDV',
 'SGP',
 'DIE RECHTE',
 'Tierschutzallianz',
 'B*',
 'Die Grauen',
 'du',
 'MG',
 'MENSCHLICHE WELT',
 'Gesundheitsforschung',
 'V-Partei³',
 'Bündnis C',
 'DIE EINHEIT',
 'MIETERPARTEI',
 'Neue Liberale']
btw_es['Andere'] = btw_es[andere].sum(axis=1) # sum votes for 'Other' Parties
union = ['CDU', 'CSU']
btw_es['Union'] = btw_es[union].sum(axis=1) # sum votes for 'Union' Parties
linke = ['Die Linke', 'DIE LINKE', 'PDS']
btw_es['Linke'] = btw_es[linke].sum(axis=1) # sum votes for 'Linke' Parties

ZS: Sum votes for other, Union and DIE LINKE parties

In [ ]:
#list(btw_zs.columns)

In [ ]:
andere = ['AfD',
 'REP',
 'GRAUE',
 'Tierschutz',
 'NPD',
 'ödp',
 'PBC',
 'FRAUEN',
 'BP',
 'FAMILIE',
 'CM',
 'BüSo',
 'HP',
 'Violetten',
 'AUFBRUCH',
 'ZENTRUM',
 'KPD',
 'PRG',
 'Schill',
 'Offensive D',
 'Die Tierschutzpartei',
 'DIE FRAUEN',
 'Deutschland',
 'AGFG',
 'APPD',
 '50plus',
 'MLPD',
 'Die Partei',
 'PSG',
 'Pro DM',
 'Volksabstimmung',
 'ADM',
 'DKP',
 'DVU',
 'DIE VIOLETTEN',
 'FWD',
 'PIRATEN',
 'RRP',
 'RENTNER',
 'Tierschutzpartei',
 'ÖDP',
 'Bündnis 21/RRP',
 'BIG',
 'pro Deutschland',
 'DIE RECHTE',
 'FREIE WÄHLER',
 'Partei der Nichtwähler',
 'PARTEI DER VERNUNFT',
 'Die PARTEI',
 'PDV',
 'SGP',
 'AD-DEMOKRATEN',
 'Tierschutzallianz',
 'B*',
 'BGE',
 'DiB',
 'DM',
 'Die Grauen',
 'du.',
 'MG',
 'MENSCHLICHE WELT',
 'Die Humanisten',
 'Gesundheitsforschung',
 'V-Partei³']
btw_zs['Andere'] = btw_zs[andere].sum(axis=1) # sum votes for 'Other' Parties
union = ['CDU', 'CSU']
btw_zs['Union'] = btw_zs[union].sum(axis=1) # sum votes for 'Union' Parties
linke = ['Die Linke', 'DIE LINKE', 'PDS']
btw_zs['Linke'] = btw_zs[linke].sum(axis=1) # sum votes for 'Linke' Parties

Concatenate Erst- and Zweitstimme datasets

In [ ]:
dfs = [btw_es, btw_zs]
btw = pd.concat(dfs, sort=False)
btw = btw.sort_values(['RS', 'year'])
btw.head(10)

Add post-treatment indicator

In [ ]:
years = [2013, 2017]
for year in years:
    btw[f'post_{year}'] = np.where(btw['year'] >= year, 1, 0)
btw.head(10)

Set index

In [ ]:
#btw = btw.set_index(['AGS', 'year'])
btw.head(10)

Drop not needed columns and rename

In [ ]:
btw = btw[['RS', 'year', 'post_2013', 'post_2017', 'Land', 'Erststimme', 'Zweitstimme', 'Wahlberechtigte (A)', 'Wähler (B)', 'Union', 'SPD', 'FDP', 'Linke', 'GRÜNE', 'Andere']]
btw.rename(columns = {'Wahlberechtigte (A)':'Wahlberechtigte', 'Wähler (B)': 'Wähler', 'GRÜNE': 'Grüne'}, inplace = True)
btw.head(10)

Calculate vote shares in percent

In [ ]:
parties = ['Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere']
for party in parties:
    btw.loc[:, party] = (btw[party] / btw['Wähler'])*100
btw.loc[:, 'Wähler'] = (btw['Wähler'] / btw['Wahlberechtigte'])*100
btw.head(10)

Merge with treatment dataset

In [ ]:
treatment = pd.read_csv(f'{path}/data/treatment.csv', converters = {'RS': str})
#data = btw.join(treatment.set_index(['AGS']))
data = pd.merge(btw, treatment, on='RS', how='left', indicator=True)
data.head(10)

Inspect merge results

In [ ]:
print(data['_merge'].describe())

In [ ]:
#not_merged = data[data['_merge'] == 'both']
not_merged = data[data['_merge'] != 'both']

not_merged = not_merged.reindex(columns = ['RS'])

not_merged['Land'] = not_merged['RS'].str.slice(start=0, stop=2)
not_merged['Regierungsbezirk'] = not_merged['RS'].str.slice(start=2, stop=3)
not_merged['Kreis'] = not_merged['RS'].str.slice(start=3, stop=5)
not_merged['VG'] = not_merged['RS'].str.slice(start=5, stop=9)
not_merged['Gemeinde'] = not_merged['RS'].str.slice(start=9, stop=12)

not_merged = not_merged.groupby(['RS'], as_index=True).first()
#not_merged['merge'] = True
not_merged['merge'] = False
not_merged.to_csv(f'{path}/data/not_merged.csv')
not_merged.head()

Clean data

In [ ]:
data = data.set_index(['AGS', 'year'])
data = data.drop(columns=['_merge'])
data = data.reindex(columns=['post_2013', 'post_2017', 'GEN', 'Land', 'Wahlberechtigte', 'Wähler', 'Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere', 'Erststimme', 'Zweitstimme', 'treated_0', 'treated_10', 'treated_20', 'treated_30', 'treated_50', 'treated_100'])
#data = data.sort_values(by=['AGS', 'year'])
data.head(10)

Interact treatment dummies with post-treatment indicators

In [ ]:
dummies = ['treated_0', 'treated_10', 'treated_20', 'treated_30', 'treated_50', 'treated_100']
for dummy in dummies:
    data.insert(loc=len(data.columns), column=f'{dummy}*post_2013', value=(data[dummy]*data['post_2013']))
    data.insert(loc=len(data.columns), column=f'{dummy}*post_2017', value=(data[dummy]*data['post_2017']))
data.head(10)

Drop missing values

In [ ]:
data = data.dropna(axis='rows', how='any', subset=['GEN', 'Union',	'SPD', 'FDP', 'Linke', 'Grüne', 'Andere'])

Export dataset

In [ ]:
data.to_csv(f'{path}/data/btw_treatment.csv', encoding = 'utf-8-sig')